In [1]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

!pip install -qq -U langchain tiktoken faiss-gpu
!pip install -qq -U transformers InstructorEmbedding sentence_transformers
!pip install -qq -U accelerate bitsandbytes xformers einops
!pip install replicate
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
ERROR: pip's d

In [2]:
import requests
import difflib
from langchain.document_loaders import GutenbergLoader
from langchain.document_loaders import WikipediaLoader
import os
import uvicorn

import langchain
from fastapi import FastAPI

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain import PromptTemplate, ConversationChain, LLMChain
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma, FAISS

from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

from langchain.chains import RetrievalQA, ConversationalRetrievalChain

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


from langchain.llms import Replicate
from langchain import PromptTemplate, LLMChain

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [33]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import textwrap

In [4]:
app = FastAPI()

In [5]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [6]:
book_embeddings = None
book_content = None

In [7]:
class Configuration:
    model_name = "llama2-13b"
    temperature = 0.5
    top_p = 0.95
    repetition_penalty = 1.15

    split_chunk_size = 1000
    split_overlap = 100

    embeddings_model_repo = "hkunlp/instructor-large"

    k = 3

    Persist_directory = "./embeddings/"

In [8]:
def remove_project_gutenberg_sections(text):
    start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
    end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"
    start_index = text.find(start_marker)
    start_end_index = text.find("***", start_index + len(start_marker))
    end_index = text.find(end_marker)
    text = text[start_end_index + 3 : end_index]
    return text

In [35]:
def does_book_exist(book_id):
    global book_embeddings
    print("Checking for existing instructor embeddings...")
    try:
        embedding_name = f"BabblerEmbedding-{book_id}"
        book_embeddings = FAISS.load_local(
            embedding_name,
            instructor_embeddings
        )
        return True
    except:
        return False

In [10]:
def select_book(book_id, book_name):
    formatted_url = f"https://www.gutenberg.org/cache/epub/{book_id}/pg{book_id}.txt"
    print(formatted_url)
    loader = GutenbergLoader(formatted_url)
    docs = WikipediaLoader(query=book_name, load_max_docs=1).load()[0].page_content
    book_content = loader.load()[0].page_content
    filtered_book_content = remove_project_gutenberg_sections(book_content)
    filtered_book_content = docs + filtered_book_content
    print("Book content loaded.")
    return [
        Document(page_content=filtered_book_content, metadata={"source": formatted_url})
    ]

In [36]:
def create_book_embeddings(book_content, book_id: str):
    global book_embeddings
    print("Creating instructor embeddings...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=Configuration.split_chunk_size,
        chunk_overlap=Configuration.split_overlap,
    )
    print("Book content split into chunks.")
    texts = text_splitter.split_documents(book_content)
    print("Creating book embeddings...")
    book_embeddings = FAISS.from_documents(
        documents=texts,
        embedding=instructor_embeddings,
    )
    print("Book embeddings created.")
    embedding_name = f"BabblerEmbedding-{book_id}"
    book_embeddings.save_local(
       embedding_name
    )
    print("Book embeddings saved.")

In [12]:
def wrap_text_preserve_newlines(text, width=200):  # 110
    # Split the input text into lines based on newline characters
    lines = text.split("\n")

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = "\n".join(wrapped_lines)

    return wrapped_text

In [13]:
def wrap_text_preserve_newlines(text, width=200):  # 110
    # Split the input text into lines based on newline characters
    lines = text.split("\n")

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = "\n".join(wrapped_lines)

    return wrapped_text

In [39]:
def process_llm_response(llm_response):
    print(llm_response)
    ans = wrap_text_preserve_newlines(llm_response["result"])
    # sources_used = " \n".join(
    #     [str(source.metadata["source"]) for source in llm_response["source_documents"]]
    # )
    ans = ans #+ "\n\nSources: \n" + sources_used
    return ans

In [15]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


In [40]:
def generate_answer_from_embeddings(query, book_id: str):
    """
    Retrieve documents from the vector database and then pass them to the language model to generate an answer.

    Args:
        query: The user's question.
        book_id: The id of the book to search.

    Returns:
        The answer to the question.
    """
    # select books embeddings from database.
    # book_embeddings = Chroma(
    #     persist_directory=Configuration.Persist_directory,
    #     collection_name=f"BabblerEmbedding-{book_id}",
    #     embedding_function=instructor_embeddings,
    # )
    # print(f"Loaded book embeddings from {book_embeddings._collection.name}")
    retriever = book_embeddings.as_retriever(
        search_kwargs={"k": Configuration.k, "search_type": "similarity"}
    )
    # docs = book_embeddings.similarity_search(query)
    # a = [doc.page_content for doc in docs]
    # for i in a:
    #     print(i)
    # return "lorem ipsum"
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=False,
        verbose=False,
    )
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)

    return ans

In [23]:
REPLICATE_API_TOKEN = "r8_KWM7ZPHF27SufFBDWyTQdAHvU07aUHm2aUjQh"
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
NGROK_API_TOKEN = "2XUqYKO0EBFXTtX1j4wweNw0AXX_7LZayy8Lh149n7DFFiQTe"

In [29]:
!ngrok authtoken 2XUqYKO0EBFXTtX1j4wweNw0AXX_7LZayy8Lh149n7DFFiQTe

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [18]:
llm = Replicate(
    model="meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d",
    input={"temperature": 0.75, "max_length": 500, "top_p": 0.95, "repetition_penalty": 1.15},
)

In [19]:
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name=Configuration.embeddings_model_repo, model_kwargs={"device": "cuda"}
)


load INSTRUCTOR_Transformer
max_seq_length  512


In [38]:
@app.get("/book")
async def get_book(book_id: str, book_name: str):
    torch.cuda.empty_cache()
    has_book = does_book_exist(book_id)
    if has_book:
        return {"status": "success"}

    print("Getting book...")
    book_content = select_book(book_id, book_name)
    create_book_embeddings(book_content, book_id)
    return {"status": "success"}

In [21]:
@app.get("/answer")
async def get_answer(query: str, book_id: str):
    has_book = does_book_exist(book_id)
    if not has_book:
        return {"status": "error", "message": "Book not found."}
    return generate_answer_from_embeddings(query, book_id)

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://8c33-35-185-138-88.ngrok-free.app


INFO:     Started server process [1079]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2402:4000:2081:27dd:e547:1ae2:e5a4:2b0c:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2402:4000:2081:27dd:e547:1ae2:e5a4:2b0c:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2402:4000:2081:27dd:e547:1ae2:e5a4:2b0c:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2402:4000:2081:27dd:e547:1ae2:e5a4:2b0c:0 - "GET /openapi.json HTTP/1.1" 200 OK
Checking for existing instructor embeddings...
INFO:     2402:4000:2081:27dd:e547:1ae2:e5a4:2b0c:0 - "GET /book?book_id=1342&book_name=Pride%20and%20Prejudice HTTP/1.1" 200 OK


In [ ]:















# def create_book_embeddings(book_content, book_id: str):
#     global book_embeddings
#     print("Creating instructor embeddings...")
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=Configuration.split_chunk_size,
#         chunk_overlap=Configuration.split_overlap,
#     )
#     print("Book content split into chunks.")
#     texts = text_splitter.split_documents(book_content)

#     persistent_client = chromadb.PersistentClient(Configuration.Persist_directory)
#     print("Creating book embeddings...")
#     book_embeddings = Chroma(
#         collection_name=f"BabblerEmbedding-{book_id}",
#         embedding_function=instructor_embeddings,
#         client=persistent_client,
#         persist_directory=Configuration.Persist_directory,
#     )

#     # try:
#     #     book_embeddings = Chroma.load(persist_directory = '.',
#     #                            collection_name = 'book')
#     # except:
#     # book_embeddings = Chroma.from_documents(
#     #     documents=texts,
#     #     embedding=instructor_embeddings,
#     #     persist_directory="./embeddings/",
#     #     collection_name="book",
#     # )

#     book_embeddings.add_documents(documents=texts)
#     book_embeddings.persist()
#     print("Book embeddings created.")












# def generate_answer_from_embeddings(query, book_id: str):
#     """
#     Retrieve documents from the vector database and then pass them to the language model to generate an answer.

#     Args:
#         query: The user's question.
#         book_id: The id of the book to search.

#     Returns:
#         The answer to the question.
#     """
#     # select books embeddings from database.
#     book_embeddings = Chroma(
#         persist_directory=Configuration.Persist_directory,
#         collection_name=f"BabblerEmbedding-{book_id}",
#         embedding_function=instructor_embeddings,
#     )
#     print(f"Loaded book embeddings from {book_embeddings._collection.name}")
#     retriever = book_embeddings.as_retriever(
#         search_kwargs={"k": Configuration.k, "search_type": "similarity"}
#     )
#     docs = book_embeddings.similarity_search(query)
#     # a = [doc.page_content for doc in docs]
#     # for i in a:
#     #     print(i)
#     # return "lorem ipsum"
#     qa_chain = RetrievalQA.from_chain_type(
#         llm=llm,
#         chain_type="stuff",
#         retriever=retriever,
#         chain_type_kwargs={"prompt": PROMPT},
#         return_source_documents=True,
#         verbose=False,
#     )
#     llm_response = qa_chain(query)
#     ans = process_llm_response(llm_response)

#     return ans





